# Archive statistics
## Completeness

In [1]:
import ipywidgets as widgets
widgets.Widget.widget_types
import requests
import geopy
import pandas as pd
from IPython.display import display
%matplotlib inline

import seaborn as sns
import plotly
from ipywidgets import interact

from geopy.geocoders import Nominatim
geolocator = Nominatim(timeout=3)

from IPython.display import clear_output

import json
import datacube

dc = datacube.Datacube(app='dc-example')

In [2]:
products = dc.list_products()['description'].tolist()
products_name = dc.list_products()['name'].tolist()

In [3]:
# Create a dict to allow product picker with description
products_name = dc.list_products()['name'].tolist()
product_dict = {}
for idx, val in enumerate(products):
    product_dict[val]= products_name[idx]

In [4]:
from __future__ import print_function

from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

from traitlets import link

In [5]:
aoi_picker = widgets.Text(value='Canberra',placeholder='update this field',
    description='placename search',
    disabled=False)
def handle_submit(sender):
    global location
    global center
    global zoom
    location = geolocator.geocode(aoi_picker.value)
    center = [location.latitude, location.longitude]
    zoom = 13
    print(center)
aoi_picker.on_submit(handle_submit)

In [6]:
#TODO Replace with datapicker widget when ipywidgets devs sort this out
from_date_picker = widgets.Text(value='2000-12-31',placeholder='update this field',
    description='start date',
    disabled=False)
def handle_from_date(sender):
    print(from_date_picker.value)
from_date_picker.on_submit(handle_from_date)

In [7]:
#TODO Replace with datapicker widget when ipywidgets devs sort this out
to_date_picker = widgets.Text(value='2001-01-01',placeholder='update this field',
    description='end date',
    disabled=False)
def handle_to_date(sender):
    print(to_date_picker.value)
to_date_picker.on_submit(handle_to_date)

In [8]:
product_picker = widgets.RadioButtons(
    options=products,
    description='product:',
    disabled=False
)

In [9]:
zoom_picker =widgets.RadioButtons(
    options=['city','regional'],
    description='zoom:',
    disabled=False
)

In [14]:
stats_picker =widgets.RadioButtons(
    options=['count files with bounds intersecting AOI','count all files'],
    description='zoom:',
    disabled=False
)

In [15]:
accordion = widgets.Accordion(children=[product_picker,from_date_picker, to_date_picker, aoi_picker,zoom_picker,stats_picker ])
accordion.set_title(0, 'product')
accordion.set_title(1, 'from date')
accordion.set_title(2, 'to date')
accordion.set_title(3, 'placename')
accordion.set_title(4, 'zoom level')
accordion.set_title(5, 'statistic')
accordion

[-35.2975905, 149.1012676]


In [16]:
dc = DrawControl(polygon={'shapeOptions': {'color': '#0000FF'}}, polyline={'shapeOptions': {'color': '#00FF00'}},
                      circle={'shapeOptions': {'color': '#0000FF'}}, rectangle={'shapeOptions': {'color': '#0000FF'}})
# add the DrawControl to the map
m = Map(center=center, zoom=zoom)
m.add_control(dc)
m

In [17]:
# initialise the DrawControl to capture the geometry
update_map = widgets.Button(description="Update map on click")
m = None
def update_map_clicked(button):
    global m
    global zoom
    global dc
    if not m:
        dc = DrawControl(polygon={'shapeOptions': {'color': '#0000FF'}}, polyline={'shapeOptions': {'color': '#00FF00'}},
                      circle={'shapeOptions': {'color': '#0000FF'}}, rectangle={'shapeOptions': {'color': '#0000FF'}})
        # add the DrawControl to the map
        m = Map(center=center, zoom=zoom)
        m.add_control(dc)
        display(m)
    else:
        m.center = center
display(update_map)
update_map.on_click(update_map_clicked)

In [20]:
button = widgets.Button(description="Generate Report")
def on_button_clicked(button):
    print('Generating your report now, hold onto your shorts')

    #print(product_picker.value, sensor_picker.value, from_date_picker.value, to_date_picker.value, dc.last_draw['geometry'])
    
    if (stats_picker.value == 'count all files'):
        product = product_dict[product_picker.value]

        dc = datacube.Datacube()
        gw = datacube.api.GridWorkflow(dc.index, product=product)
        cells = gw.list_cells(product=product, time=(from_date_picker.value, to_date_picker.value))

        for idx, data in cells.items():
            print(idx, data.shape[0])
    else:
        print('No statistic selected')
    
display(button)
button.on_click(on_button_clicked)

Generating your report now, hold onto your shorts


In [ ]:
# Example of tabs - ignore for now
#list = ['P0', 'P1', 'P2', 'P3', 'P4']
#children = [accordion,update_map]
#print(children)
#tab = widgets.Tab(children=children)
#tab

ul_lon, ul_lat = 77.83, 17.84 # upper left longitude, latitude
lr_lon, lr_lat = 78.00, 17.67 # lower right longitude, latitude
date_start, date_end = from_date_picker.value, to_date_picker.value  # start date and end date for time range selection  
sensor_list=["olitirs8", "tm5", "etm7"] # list of sensors to use

polygon_list = [[ul_lon, ul_lat], [lr_lon, ul_lat],[lr_lon, lr_lat],[ul_lon, lr_lat],[ul_lon, ul_lat]]

wrs_query = 'http://api.remotepixel.ca/wrs2tiles?search=poly:'+str(polygon_list)

post_query = requests.get(wrs_query)
wrs_search_result = json.loads(post_query.text)

path_row = []
for item in wrs_search_result['results']:
    path_row.append(str(item['path'])+"_"+str(item['row']))
path_row

tips = sns.load_dataset('tips')